<a href="https://colab.research.google.com/github/keulreobeu/deep_learning_for_the_life_sciences/blob/main/%EC%83%9D%EB%AC%BC%EB%AC%BC%EB%A6%AC%ED%95%99%EA%B3%BC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5. 생물물리학과 머신러닝

*   단백질 구조 예측
    1.   상동성 모델링: 구조가 알려진 단백질의 상동성 단백질을 찾아 조정을 통해 상동성 단백질의 구조를 예측, 조건이 어렵다
    2.   물리적 모델링: 물리법칙에 따라 안정적인 구조를 예측, 엄청난 계산이 필요, 항상 맞지 않음

*   생물물리학적 피처화
    1.   그리드 피쳐화: 단백질의 수소결합과 이온결합과 같은 상호작용을 3차원 구조에 명시적으로 나타낸 기법, 새로운 물리학적 정보는 찾지 못한다
    2.   원자 피처화: 모든 원자를 식별자와 3차원 좌표로 표현한 데이터로 나타냄, 물리적 상호작용을 방법을 스스로 배워야 하기 때문에 계산량이 많다는 단점, 하지만 알려지지 않은 상호작용을 찾아낼 수 있음

*   그리드 피처화
    *   수소결합
        -  단백질은 주로 비공유 결합에 의해 결정되기 때문에 이러한 특성을 살린 피쳐화를 사용해야 한다.
        -   RdkitGridFeaturizer는 원자들 중 충분히 가까운거리에 있는 수소 결합의 숫자를 파악, 거리에 대한 컷오프 값이 필요하다.

    *   이온결합
        -   일반적으론 두 아미노산 사이에 일어나는 비공유 결합
        -   3차원 구조에서 근접한 모든 아미노산의 쌍을 검사해 이온 결합 여부를 확인하다.

    *   파이 겹침 결합
        -   방향족 고리 간 비공유 결합
        -   서로 면이 맞닫고 있는 센드위치 형태, 서로 맞댄 형태, 비스듬하게 겹친 형태 등 여러 겹침이 있다.
        -   N개의 원자를 가지는 분자는 (N, 3)배열로 나타내고, 화학 결합을 찾ㄱ기 위해 충분히 근접한 원자 쌍들을 추려낸다. 예를들면 파이 겹침의 경우 먼저 방향족 고리를 찾은 후 그 고리간의 중심을 계산해 충부히 근접한 원자 쌍을 찾는다.

*   원자 피쳐화
    *   분자의 3차원 구조를 나타내는 넌파이의 (N, 3)배열은 각 위치의 원자의 정보가 없다. 그러므로 각 위치의 원자번호를 나열하는 또다른 배열이 필요하다.
    *   원자 사이의 거리 계산량을 줄이기 위해 사전에 이웃 원자 목록을 만든다. 거리계산량을 줄이기 위해 가까운 원자를 목록화한 것
    *  dc.feat.ComplexNeighborListFragmentAtomicCoordinat 기능을 통해 재공한다.

        

# 생물물리학 데이터 사례 연구

*   PDBBind 데이터셋
    *   생체 분자의 3차원 구조와 이들의 결합 친화력을 포함하고 있는 데이터셋
    *   단백질 이외에도 DNA, RNA 같은 핵산, 지질 등이 포함된다.
    *   결합 친화력은 실험을 통해 얻어진 두 분자의 친화력 이며, 클수록 두 분자가 복합체 형태로 더 많이 존재한다는 뜻이다.
    *   대부분 단백질 - 리간드 복합체 이지만 단백질 - 단백질 또는 단백질 - DNA, DNA - 리간드 등 여러 복합채들이 포함돼 있다
    *   각 복합체에 결합 친화력에 대한 실험값이 주석으로 처리돼 있다.
    *   주로 단백질 정보 은행에서 수집된 정보로 만들어 졌는데, 이는 실험에 대한 설정과 측정값에 실험적 차이가 있어 결과적으론 데이터 품질이 일정하지 못하다.


---

# PDBBind 데이터셋 피처화

25.07 구버전 런타임 실행

In [1]:
!pip install -U pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [1]:
!pip install --pre deepchem

런타임 재시작 후 다시실행


In [1]:
!pip install "keras<3" "tensorflow<2.16"

런타임 재시작 후 다시실행

In [4]:
!pip install rdkit

KeyboardInterrupt: 

In [5]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 84.1 MB/s  0:00:00


In [6]:
import deepchem as dc
from rdkit import Chem
import numpy as np
import pandas as pd

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [7]:
'''
sanitize -> Deepchem이 탐치한 명백한 오류를 수정하는 옵션
flatten -> 각각의 입력 구조에 대해 1차원 피처를 출력
feature_types -> 입력 데이터에서 감지하려는 생물물리학적 특성과 화학 결합의 유형 설정
voxel_width -> 그리드를 구성하는 복셀의 크기 2.0은 2옴스트롬
단백질을 복셀화 표현으로 변환
각 복셀에 대해 생물물리학적 특성을 계산하고 부분 벡터도 계산한다.
ECFP(확장 연결 지문) 그리고 SPLIF 두가지 화학 지문을 계산하다.
'''

grid = dc.feat.RdkitGridFeaturizer(
    voxel_width=2.0,
    feature_types=['hbond', 'salt_bridge','pi_stack', 'cation_pi', 'ecfp', 'splif'],
    sanitize=True, flatten=True
)

In [8]:
# 2.6 이후의 deepchem은 featurizer에서 'grid'를 지원하지 않음

tasks, dataset, transformers = dc.molnet.load_pdbbind(
    featurizer='ecfp',
    split='random',
    subset='core'
)
train_dataset, valid_dataset, test_dataset = dataset

    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFinger

In [8]:
from sklearn.ensemble import RandomForestRegressor

sklearn_model = RandomForestRegressor(n_estimators=100)
model = dc.models.SklearnModel(sklearn_model)
model.fit(train_dataset)

ValueError: Found array with 0 feature(s) (shape=(154, 0)) while a minimum of 1 is required by RandomForestRegressor.

In [12]:
n_features = train_dataset.X.shape[1]
model = dc.models.MultitaskRegressor(
    n_tasks=len(tasks),
    n_features=n_features,
    layer_sizes=[2000, 1000],
    dropouts=0.5,
    learning_rate=0.003
)
model.fit(train_dataset, nb_epoch=50)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [16]:
import deepchem as dc
from deepchem.metrics import Metric, mae_score, rms_score

# 1) 데이터 로드 (split -> splitter로 교체 권장)
tasks, dataset, transformers = dc.molnet.load_pdbbind(
    featurizer='ecfp',        # 현재 벡터 피처 사용
    splitter='random',        # 'split' 경고 방지
    subset='core',
    reload=False,             # 캐시 이슈 있으면 False 권장
    save_dir=None
)
train_dataset, valid_dataset, test_dataset = dataset

# 2) 모델 구성: MultitaskRegressor (MLP)
n_tasks = len(tasks)                   # PDBBind는 보통 1개 task
n_features = train_dataset.X.shape[1]  # 입력 차원

model = dc.models.MultitaskRegressor(
    n_tasks=n_tasks,
    n_features=n_features,
    layer_sizes=[2000, 1000],   # 은닉층 너비 2000, 1000
    dropouts=[0.5, 0.5],        # 각 은닉층 드롭아웃 0.5
    learning_rate=1e-3,         # 필요시 조정
    batch_size=64,              # 필요시 조정
)

# 3) 학습
model.fit(train_dataset, nb_epoch=50)  # 필요시 epoch 조정

# 4) 평가 (MAE, RMSE)
# mode='regression'을 명시
mae = Metric(mae_score, mode='regression', name='MAE')
rmse = Metric(rms_score, mode='regression', name='RMSE')

valid_scores = model.evaluate(valid_dataset, [mae, rmse])
test_scores  = model.evaluate(test_dataset, [mae, rmse])

print("Valid MAE:", valid_scores['MAE'])
print("Valid RMSE:", valid_scores['RMSE'])
print("Test MAE:", test_scores['MAE'])
print("Test RMSE:", test_scores['RMSE'])


    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFingerprintAsBitVect(RDKit::ROMol mol, unsigned int radius, unsigned int nBits=2048, boost::python::api::object invariants=[], boost::python::api::object fromAtoms=[], bool useChirality=False, bool useBondTypes=True, bool useFeatures=False, boost::python::api::object bitInfo=None, bool includeRedundantEnvironments=False)
    rdkit.Chem.rdMolDescriptors.GetMorganFingerprintAsBitVect(tuple, int)
did not match C++ signature:
    GetMorganFinger

Valid MAE: 0.9700430482595526
Valid RMSE: 1.158068459533841
Test MAE: 0.6429874374636814
Test RMSE: 0.8006818588912367
